# Продвинутое использование Qdrant для RAG

В этом блокноте мы продолжим изучение Qdrant и шаг за шагом построим минимальный pipeline Retrieval Augmented Generation (RAG).

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance
from sentence_transformers import SentenceTransformer
from transformers import pipeline

client = QdrantClient(':memory:')
encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

## Шаг 1. Подготовка данных и индексирование

In [ ]:
texts = [
    "Qdrant это векторная база данных с открытым исходным кодом.",
    "Она обеспечивает быстрый поиск по большим наборам векторов.",
    "Qdrant отлично подходит для проектов с использованием нейронных сетей.",
    "Хранилище поддерживает фильтрацию по метаданным.",
    "RAG комбинирует поиск документов и генерацию ответа."
]
embeddings = encoder.encode(texts)
collection_name = 'rag_tutorial'
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=len(embeddings[0]), distance=Distance.COSINE)
)
points = [PointStruct(id=i, vector=vec, payload={'text': text}) for i, (vec, text) in enumerate(zip(embeddings, texts))]
client.upsert(collection_name=collection_name, points=points)

## Шаг 2. Поиск релевантных документов

In [ ]:
query = 'Для чего нужен Qdrant?'
query_vec = encoder.encode(query)
hits = client.search(collection_name=collection_name, query_vector=query_vec, limit=3)
hits

## Шаг 3. Генерация ответа на основе найденных документов

In [ ]:
context = '
'.join(hit.payload['text'] for hit in hits)
qa_prompt = f'Вопрос: {query}
Документы:
{context}
Ответ:'
text_generator = pipeline('text-generation', model='gpt2')
answer = text_generator(qa_prompt, max_new_tokens=50)[0]['generated_text']
print(answer)

## Шаг 4. Дополнительные возможности

В Qdrant можно использовать фильтры, настраивать параметры поиска и хранить произвольные метаданные для более сложных сценариев.

In [ ]:
# Пример 1. Поиск с простым фильтром по метаданным
filter_ = Filter(must=[FieldCondition(key='category', match=MatchValue(value='article'))])
hits = client.search(collection_name=collection_name, query_vector=query_vec, filter=filter_, limit=5)
for hit in hits:
    print(hit.id, hit.payload)


In [ ]:
# Пример 2. Сложный фильтр AND/OR
complex_filter = Filter(must=[FieldCondition(key='lang', match=MatchValue(value='ru'))],
                         should=[FieldCondition(key='rating', range=Range(gte=4))])
hits = client.search(collection_name=collection_name, query_vector=query_vec, filter=complex_filter, limit=3)
for hit in hits:
    print(hit.id, hit.payload)


In [ ]:
# Пример 3. Фильтрация по диапазону числового значения
price_filter = Filter(must=[FieldCondition(key='price', range=Range(gte=10, lte=20))])
hits = client.search(collection_name=collection_name, query_vector=query_vec, filter=price_filter, limit=3)
for hit in hits:
    print(hit.id, hit.payload['price'])


In [ ]:
# Пример 4. Использование параметров поиска для повышения точности
params = SearchParams(hnsw_ef=256)
hits = client.search(collection_name=collection_name, query_vector=query_vec, limit=3, search_params=params)


In [ ]:
# Пример 5. Сохранение произвольных метаданных
payload = {'text': 'пример текста', 'lang': 'ru', 'category': 'article'}
point = PointStruct(id=123, vector=encoder.encode(payload['text']).tolist(), payload=payload)
client.upsert(collection_name=collection_name, points=[point])


In [ ]:
# Пример 6. Получение точек по фильтру
points = client.scroll(collection_name=collection_name, filter=filter_, limit=10)[0]
print('Найдено', len(points), 'точек')


In [ ]:
# Пример 7. Обновление метаданных для точек
client.set_payload(collection_name=collection_name, payload={'rating': 5}, points=[123])


In [ ]:
# Пример 8. Удаление точек по фильтру
client.delete(collection_name=collection_name, filter=complex_filter)


In [ ]:
# Пример 9. Подсчет точек по фильтру
count = client.count(collection_name=collection_name, filter=filter_, exact=True).count
print('Всего', count)


In [ ]:
# Пример 10. Рекомендации схожих элементов
recommendations = client.recommend(collection_name=collection_name, positive=[123], limit=5)
for hit in recommendations:
    print(hit.id)


In [ ]:
# Пример 11. Итерация по результатам с помощью scroll
offset = None
while True:
    points, offset = client.scroll(collection_name=collection_name, offset=offset, limit=50)
    if not points:
        break
    process(points)


In [ ]:
# Пример 12. Пакетный поиск для нескольких запросов
requests = [SearchRequest(vector=encoder.encode(q).tolist(), limit=3) for q in queries]
results = client.search_batch(collection_name=collection_name, requests=requests)


In [ ]:
# Пример 13. Гибридный поиск с использованием BM25S
bm25_params = BM25SearchParams()
hybrid_results = client.search(collection_name=collection_name, query_vector=query_vec, query_text=query, search_params=bm25_params, limit=5)


In [ ]:
# Пример 14. Настройка весов для векторной и текстовой частей при гибридном поиске
bm25_params = BM25SearchParams(alpha=0.7)  # вес текстовой компоненты
hybrid_results = client.search(collection_name=collection_name, query_vector=query_vec, query_text=query, search_params=bm25_params, limit=5)


In [ ]:
# Пример 15. Работа с результатами гибридного поиска
for hit in hybrid_results:
    print(hit.id, hit.payload.get('text'))
